In [1]:
import dash

from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

year_r = 2023
year_r2 = 2022

df = pd.read_csv(f'grouped/{year_r}/grouped_by_subcat_{year_r}.csv', encoding='UTF-8')
df["YEAR"] = year_r
df2 = pd.read_csv(f'grouped/{year_r2}/grouped_by_subcat_{year_r2}.csv', encoding='UTF-8')
df2["YEAR"] = year_r2

app = dash.Dash(__name__)

min_week = df['Week number'].min()
max_week = df['Week number'].max()

app.layout = html.Div([
    html.H1("Dashboard", className='dashboard-title'),
    
    html.Label("Выберите категории", className='white-background'),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': cat, 'value': cat} for cat in df['CATEGORY'].unique()],
        value=[df['CATEGORY'].unique()[0]], 
        multi=True  
    ),
    html.Label("Выберите подкатегории"),
    dcc.Dropdown(id='subcategory-dropdown', multi=True),
    
    html.Label("Выберите диапазон недель", className='white-background'),
    dcc.RangeSlider(
        id='week-range-slider',
        min=min_week,
        max=max_week,
        step=1,
        value=[min_week, max_week],
        marks={i: str(i) for i in range(min_week, max_week+1)}
    ),
    
    html.Div([
        dcc.Checklist(
            id='group-by-category',
            options=[{'label': 'Группировать по категории', 'value': 'group'}],
            value=[],
            className='white-background-checkbox'
        )
    ], className='white-background-checkbox'),
    
    dcc.Graph(id='weekly-graph-2022'),
    dcc.Graph(id='weekly-graph'),
    dcc.Graph(id='bonus-graph'),
    
    html.Hr(), 
    
    html.H2("Seasonality Analysis", className='dashboard-title'),
    dcc.Graph(id='weekly-comparison-graph'),
    dcc.Graph(id='seasonality-weekday'),
    dcc.Graph(id='seasonality-month'),
    dcc.Graph(id='trend-decomposition-trend'),
    dcc.Graph(id='trend-decomposition-seasonal'),
    dcc.Graph(id='trend-decomposition-resid')
])

@app.callback(
    [Output('subcategory-dropdown', 'options'), Output('subcategory-dropdown', 'value')],
    [Input('category-dropdown', 'value')]
)
def update_subcategories(selected_categories):
    filtered_df = df[df['CATEGORY'].isin(selected_categories)]
    subcategories = filtered_df['SUBCATEGORY'].unique()
    return [{'label': sub, 'value': sub} for sub in subcategories], list(subcategories)

@app.callback(
    [Output('weekly-graph', 'figure'),
     Output('bonus-graph', 'figure'),
     Output('weekly-graph-2022', 'figure'),
     Output('weekly-comparison-graph', 'figure'),
     Output('seasonality-weekday', 'figure'),
     Output('seasonality-month', 'figure'),
     Output('trend-decomposition-trend', 'figure'),
     Output('trend-decomposition-seasonal', 'figure'),
     Output('trend-decomposition-resid', 'figure')],
    [Input('category-dropdown', 'value'),
     Input('subcategory-dropdown', 'value'),
     Input('group-by-category', 'value'),
     Input('week-range-slider', 'value')]
)
def update_graphs(selected_categories, selected_subcategories, group_by_category, selected_weeks):
    filtered_df = df[
        (df['CATEGORY'].isin(selected_categories)) & 
        (df['SUBCATEGORY'].isin(selected_subcategories)) & 
        (df['Week number'] >= selected_weeks[0]) & 
        (df['Week number'] <= selected_weeks[1])
    ]
    filtered_df2 = df2[
        (df2['CATEGORY'].isin(selected_categories)) & 
        (df2['SUBCATEGORY'].isin(selected_subcategories)) & 
        (df2['Week number'] >= selected_weeks[0]) & 
        (df2['Week number'] <= selected_weeks[1])
    ]
    
    if 'group' in group_by_category:
        grouped_data = filtered_df.groupby(['Week number', 'CATEGORY']).sum().reset_index()
        line_fig = px.line(grouped_data, x='Week number', y='QUANTITY', color='CATEGORY', title='Продажи по неделям (2023)')
        
        grouped_bonus_data = filtered_df.groupby(['Week number', 'CATEGORY']).sum().reset_index()
        bonus_fig = go.Figure()
        for category in grouped_bonus_data['CATEGORY'].unique():
            category_data = grouped_bonus_data[grouped_bonus_data['CATEGORY'] == category]
            bonus_fig.add_trace(go.Bar(
                x=category_data['Week number'],
                y=category_data['BONUSES_SPENT'],
                name=f'{category} - BONUSES_SPENT'
            ))
            bonus_fig.add_trace(go.Bar(
                x=category_data['Week number'],
                y=category_data['BONUSES_RECIVE'],
                name=f'{category} - BONUSES_RECIVE'
            ))
            bonus_fig.add_trace(go.Scatter(
                x=category_data['Week number'],
                y=category_data['COST (without bonuses)'],
                mode='lines+markers',
                name=f'{category} - COST (without bonuses)',
                yaxis='y2'
            ))
            
        bonus_fig.update_layout(
            barmode='group',
            title='Бонусы и стоимость без бонусов по неделям (2023)',
            xaxis_title='Week number',
            yaxis_title='Bonuses',
            yaxis2=dict(
                title='Cost (without bonuses)',
                overlaying='y',
                side='right'
            ),
            legend_title='Category'
        )

        grouped_data2 = filtered_df2.groupby(['Week number', 'CATEGORY']).sum().reset_index()
        line_fig2 = px.line(grouped_data2, x='Week number', y='QUANTITY', color='CATEGORY', title='Продажи по неделям (2022)')
        
    else:
        weekly_data = filtered_df.groupby(['Week number', 'CATEGORY', 'SUBCATEGORY']).sum().reset_index()
        line_fig = px.line(weekly_data, x='Week number', y='QUANTITY', color='SUBCATEGORY', title='Продажи по неделям (2023)')
        
        bonus_data = filtered_df.groupby(['Week number', 'CATEGORY', 'SUBCATEGORY']).sum().reset_index()
        bonus_fig = go.Figure()
        for subcategory in bonus_data['SUBCATEGORY'].unique():
            subcategory_data = bonus_data[bonus_data['SUBCATEGORY'] == subcategory]
            bonus_fig.add_trace(go.Bar(
                x=subcategory_data['Week number'],
                y=subcategory_data['BONUSES_SPENT'],
                name=f'{subcategory} - BONUSES_SPENT'
            ))
            bonus_fig.add_trace(go.Bar(
                x=subcategory_data['Week number'],
                y=subcategory_data['BONUSES_RECIVE'],
                name=f'{subcategory} - BONUSES_RECIVE'
            ))
            bonus_fig.add_trace(go.Scatter(
                x=subcategory_data['Week number'],
                y=subcategory_data['COST (without bonuses)'],
                mode='lines+markers',
                name=f'{subcategory} - COST (without bonuses)',
                yaxis='y2'
            ))

        bonus_fig.update_layout(
            barmode='group',
            title='Бонусы и стоимость без бонусов по неделям (2023)',
            xaxis_title='Week number',
            yaxis_title='Bonuses',
            yaxis2=dict(
                title='Cost (without bonuses)',
                overlaying='y',
                side='right'
            ),
            legend_title='Subcategory'
        )

        weekly_data2 = filtered_df2.groupby(['Week number', 'CATEGORY', 'SUBCATEGORY']).sum().reset_index()
        line_fig2 = px.line(weekly_data2, x='Week number', y='QUANTITY', color='SUBCATEGORY', title='Продажи по неделям (2022)')
    
    comparison_df = pd.merge(
        filtered_df.groupby('Week number')['QUANTITY'].sum().reset_index().rename(columns={'QUANTITY': 'QUANTITY_2023'}),
        filtered_df2.groupby('Week number')['QUANTITY'].sum().reset_index().rename(columns={'QUANTITY': 'QUANTITY_2022'}),
        on='Week number',
        how='outer'
    ).fillna(0)
    
    comparison_fig = px.bar(
        comparison_df.melt(id_vars='Week number', value_vars=['QUANTITY_2023', 'QUANTITY_2022']),
        x='Week number', 
        y='value', 
        color='variable', 
        barmode='group',
        title='Сравнение продаж по неделям (2023 vs 2022)',
        labels={'value': 'Quantity', 'variable': 'Year'}
    )
    
    seasonality_df = df[df['CATEGORY'].isin(selected_categories)]
    seasonality_df2 = df2[df2['CATEGORY'].isin(selected_categories)]

    comparison_df_week_day = pd.merge(
        seasonality_df.groupby('Week day')['QUANTITY'].mean().reset_index().rename(columns={'QUANTITY': 'QUANTITY_2023'}),
        seasonality_df2.groupby('Week day')['QUANTITY'].mean().reset_index().rename(columns={'QUANTITY': 'QUANTITY_2022'}),
        on='Week day',
        how='outer'
    ).fillna(0)
    
    comparison_fig_week_day = px.bar(
        comparison_df_week_day.melt(id_vars='Week day', value_vars=['QUANTITY_2023', 'QUANTITY_2022']),
        x='Week day', 
        y='value', 
        color='variable', 
        barmode='group',
        title='Average Sales by Weekday (2023 vs 2022)',
        labels={'value': 'Quantity', 'variable': 'Year'}
    )

    comparison_df_month = pd.merge(
        seasonality_df.groupby('Month number')['QUANTITY'].sum().reset_index().rename(columns={'QUANTITY': 'QUANTITY_2023'}),
        seasonality_df2.groupby('Month number')['QUANTITY'].sum().reset_index().rename(columns={'QUANTITY': 'QUANTITY_2022'}),
        on='Month number',
        how='outer'
    ).fillna(0)

    comparison_fig_month = px.bar(
        comparison_df_month.melt(id_vars='Month number', value_vars=['QUANTITY_2023', 'QUANTITY_2022']),
        x='Month number', 
        y='value', 
        color='variable', 
        barmode='group',
        title='Сравнение продаж по месяцам (2023 vs 2022)',
        labels={'value': 'Quantity', 'variable': 'Year'}
    )
    comparison_fig_month.update_xaxes(tickvals=list(range(1, 13)), ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    
    seasonality_df = df[df['CATEGORY'].isin(selected_categories)].groupby(['Week number']).agg({'QUANTITY': 'sum'}).reset_index()
    seasonality_df2 = df2[df2['CATEGORY'].isin(selected_categories)].groupby(['Week number']).agg({'QUANTITY': 'sum'}).reset_index()

    all_weeks = pd.Series(np.arange(0, 52))
    missing_weeks = all_weeks[~all_weeks.isin(seasonality_df['Week number'].unique())]
    missing_weeks_df = pd.DataFrame({'Week number': missing_weeks, 'QUANTITY': 0})
    seasonality_df = pd.concat([seasonality_df, missing_weeks_df]).sort_values(by='Week number').reset_index(drop=True)

    missing_weeks = all_weeks[~all_weeks.isin(seasonality_df2['Week number'].unique())]
    missing_weeks_df2 = pd.DataFrame({'Week number': missing_weeks, 'QUANTITY': 0})
    seasonality_df2 = pd.concat([seasonality_df2, missing_weeks_df2]).sort_values(by='Week number').reset_index(drop=True)

    seasonality_df2['Week number'] += 52
    decompose_df = pd.concat([seasonality_df, seasonality_df2], axis=0)

    grouped_seasonality_df = decompose_df.groupby(['Week number']).agg({'QUANTITY': 'sum'}).reset_index()

    decomposition = seasonal_decompose(grouped_seasonality_df['QUANTITY'], model='additive', period=52)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    resid = decomposition.resid

    decomposed_df = pd.DataFrame({
        'Date': grouped_seasonality_df.index,
        'Trend': trend,
        'Seasonal': seasonal,
        'Residual': resid
    }).dropna()

    trend_fig = px.line(decomposed_df, x='Date', y='Trend', title='Trend Component', labels={'Date': 'Date', 'Trend': 'Trend'})
    seasonal_fig = px.line(decomposed_df, x='Date', y='Seasonal', title='Seasonal Component', labels={'Date': 'Date', 'Seasonal': 'Seasonal'})
    resid_fig = px.line(decomposed_df, x='Date', y='Residual', title='Residual Component', labels={'Date': 'Date', 'Residual': 'Residual'})

    return line_fig, bonus_fig, line_fig2, comparison_fig, comparison_fig_week_day, comparison_fig_month, trend_fig, seasonal_fig, resid_fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8055)
